In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

/tmp/ipykernel_140808/3777615979.py:1: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
import pandas as pd

# 1. Prepare Train Dataset

## Load the given Dataset

In [3]:
df = pd.read_csv("data/given_datasets/reco_assignment_training.csv")
df['Tran_dt'] = pd.to_datetime(df['Tran_dt'])
df.head()

,Customer_num,Tran_dt,Product_num,Tran_qty
0,C_203152,2022-01-02,P_3365,0.135
1,C_1607154,2022-01-02,P_58,1.000
2,C_84518,2022-01-02,P_21295,1.000
3,C_2553711,2022-01-02,P_20576,1.000
4,C_2376922,2022-01-02,P_1095,2.000


## Inspect the Dataset

In [4]:
#Basic information of the dataset
print("Number of purchase data: ", df.shape[0])
print("Number of features: ", df.shape[1])
print("Feature Names: ", list(df.columns))

Number of purchase data:  444404
Number of features:  4
Feature Names:  ['Customer_num', 'Tran_dt', 'Product_num', 'Tran_qty']


In [5]:
#Checking dataset for any null values
print("Does the dataset have any null values? -- {}\n".format(df.isnull().values.any()))

Does the dataset have any null values? -- False



In [6]:
#Basic information about the dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 444404 entries, 0 to 444403
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Customer_num  444404 non-null  object        
 1   Tran_dt       444404 non-null  datetime64[ns]
 2   Product_num   444404 non-null  object        
 3   Tran_qty      444404 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 13.6+ MB


In [7]:
#Statistical description of the dataset
df.describe()

,Tran_qty
count,444404.000000
mean,1.146503
std,2.206800
min,0.005000
25%,1.000000
50%,1.000000
75%,1.000000
max,1000.000000


In [8]:
#Number of duplicate entries present in the dataset
print("Number of duplicate entries present in the dataset: ",df.shape[0] - df.drop_duplicates().shape[0])

Number of duplicate entries present in the dataset:  99226


In [9]:
#Check if a customer has purchased the same product with same quantity multiple times, at multiple instances of time on the same day. 
#Manually inspected the dataset to observe that customer 'C_999487' has purchased product 'P_21270' 14 times on '2022-02-25', each time purchasing quantity 1. This may or may not be duplicate entries. 
df[(df['Customer_num'] == 'C_999487') & (df['Product_num'] == 'P_21270')]

,Customer_num,Tran_dt,Product_num,Tran_qty
19580,C_999487,2022-02-25,P_21270,1.0
56552,C_999487,2022-02-25,P_21270,1.0
118181,C_999487,2022-02-25,P_21270,1.0
167380,C_999487,2022-02-25,P_21270,1.0
217171,C_999487,2022-02-25,P_21270,1.0
229512,C_999487,2022-02-25,P_21270,1.0
253893,C_999487,2022-02-25,P_21270,1.0
315586,C_999487,2022-02-25,P_21270,1.0
328016,C_999487,2022-02-25,P_21270,1.0
364975,C_999487,2022-02-25,P_21270,1.0


In [10]:
#Check if a customer has purchased the same product with different quantity multiple times, at multiple instances of time on the same day. 
#Manually inspected the dataset to observe that customer 'C_981727' has purchased product 'P_5658' 2 times on '2022-02-20', each time purchasing a different quantity 1. 
df[(df['Customer_num'] == 'C_981727') & (df['Product_num'] == 'P_5658')]

,Customer_num,Tran_dt,Product_num,Tran_qty
31318,C_981727,2022-02-20,P_5658,0.523
278019,C_981727,2022-02-20,P_5658,0.698


## Handling duplicate entries in the dataset

Based on the above two observations on customers 'C_999487' and 'C_981727', I am assuming that these entries are actually not duplicates, but they correspond to separate purchases made by the customers at different instant of time on the same very day. Since timestamp information of purchase is not given, I have assumed that customer 'C_999487' has purchased product 'P_21270' 14 times on '2022-02-25', at different time of the day, each time purchasing the same quanity of product 'P_21270'. Similarly, 'C_981727' has made two purchases for product 'P_5658' on '2022-02-20', each time purchasing a different quantity. Based on this assumptions, I can't simply drop the duplicate entries from the dataset. Rather, I will take the aggregate sum of all the purchases made by a customer on a specific product on the same day, but at different instant of time.

#### Examples
1. Rather than having 14 same purchase entries for customer 'C_999487' purchasing product 'P_21270, I will add up the sum of the purchases made by them at different instant of time in the day. So the total quantity purchased by this customer is 14.
2. Same goes for customer 'C_981727'. Instead of taking two different purchase entries for product 'P_5658', I will add the individual quantities of purchases made by this customer for the same product. Hence, the total purchase will be 1.221 (0.523+0.698) 

In [11]:
#Creating a new column 'Total_Tran_qty', which denotes the total quantity of the same product purchased by the customer at different times on the same day.
df['Total_Tran_qty'] = df.groupby(['Customer_num', 'Tran_dt', 'Product_num'])['Tran_qty'].transform('sum')

#Dropping duplicate entries based on these columns to prepare the final data that will be used for EDA and training.
df_actual = df.drop_duplicates(subset=['Customer_num', 'Tran_dt', 'Product_num', 'Total_Tran_qty'])

#Dropping the column which we won't use further.
df_actual = df_actual.drop(['Tran_qty'], axis=1)

#Sort the dataset by time and save the data to be later used for EDA and Training Phase
df_actual.sort_values(by = ['Tran_dt', 'Customer_num', 'Product_num', 'Total_Tran_qty'],ascending=[True, True, True, True])
df_actual.to_csv('data/prepared_datasets_for_training_and_evaluation/reco_assignment_training_merged_duplicates.csv', index=None)
df_actual.head()

,Customer_num,Tran_dt,Product_num,Total_Tran_qty
0,C_203152,2022-01-02,P_3365,0.135
1,C_1607154,2022-01-02,P_58,1.000
2,C_84518,2022-01-02,P_21295,3.000
3,C_2553711,2022-01-02,P_20576,1.000
4,C_2376922,2022-01-02,P_1095,2.000


# 2. Prepare Holdout Dataset

## Load the given Dataset

In [12]:
df_holdout = pd.read_csv("data/given_datasets/reco_assignment_holdout.csv")
df_holdout['Tran_dt'] = pd.to_datetime(df['Tran_dt'])
df_holdout.head()

,Customer_num,Tran_dt,Product_num,Tran_qty
0,C_1967485,2022-01-02,P_4321,1.0
1,C_62367,2022-01-02,P_11092,1.0
2,C_2459818,2022-01-02,P_12854,1.0
3,C_851377,2022-01-02,P_9160,1.0
4,C_2351826,2022-01-02,P_17207,1.0


## Inspect the Dataset

In [13]:
#Basic information of the dataset
print("Number of purchase data: ", df_holdout.shape[0])
print("Number of features: ", df_holdout.shape[1])
print("Feature Names: ", list(df_holdout.columns))

Number of purchase data:  351397
Number of features:  4
Feature Names:  ['Customer_num', 'Tran_dt', 'Product_num', 'Tran_qty']


In [14]:
#Checking dataset for any null values
print("Does the dataset have any null values? -- {}\n".format(df_holdout.isnull().values.any()))

Does the dataset have any null values? -- False



In [15]:
#Basic information about the dataset
df_holdout.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 351397 entries, 0 to 351396
Data columns (total 4 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Customer_num  351397 non-null  object        
 1   Tran_dt       351397 non-null  datetime64[ns]
 2   Product_num   351397 non-null  object        
 3   Tran_qty      351397 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 10.7+ MB


In [16]:
#Statistical description of the dataset
df_holdout.describe()

,Tran_qty
count,351397.000000
mean,1.136690
std,1.754536
min,0.005000
25%,1.000000
50%,1.000000
75%,1.000000
max,850.000000


In [17]:
#Number of duplicate entries present in the dataset
print("Number of duplicate entries present in the dataset: ",df_holdout.shape[0] - df_holdout.drop_duplicates().shape[0])

Number of duplicate entries present in the dataset:  3738


## Handling duplicate entries in the dataset

We will follow the same methodology we followed for preparing the training dataset.

In [18]:
#Creating a new column 'Total_Tran_qty', which denotes the total quantity of the same product purchased by the customer at different times on the same day.
df_holdout['Total_Tran_qty'] = df_holdout.groupby(['Customer_num', 'Tran_dt', 'Product_num'])['Tran_qty'].transform('sum')

#Dropping duplicate entries based on these columns to prepare the final data that will be used for EDA and training.
df_holdout_actual = df_holdout.drop_duplicates(subset=['Customer_num', 'Tran_dt', 'Product_num', 'Total_Tran_qty'])

#Dropping the column which we won't use further.
df_holdout_actual = df_holdout_actual.drop(['Tran_qty'], axis=1)

#Sort the dataset by time and save the data to be later used for EDA and Training Phase
df_holdout_actual.sort_values(by = ['Tran_dt', 'Customer_num', 'Product_num', 'Total_Tran_qty'],ascending=[True, True, True, True])
df_holdout_actual.to_csv('data/prepared_datasets_for_training_and_evaluation/reco_assignment_holdout_merged_duplicates.csv', index=None)
df_holdout_actual.head( )

,Customer_num,Tran_dt,Product_num,Total_Tran_qty
0,C_1967485,2022-01-02,P_4321,1.0
1,C_62367,2022-01-02,P_11092,1.0
2,C_2459818,2022-01-02,P_12854,2.0
3,C_851377,2022-01-02,P_9160,1.0
4,C_2351826,2022-01-02,P_17207,1.0
